In [110]:
import re
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from kanjize import kanji2number
import japanize_matplotlib 

import optuna
from optuna.integration import OptunaSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

np.random.seed(42)

#行の表示数の上限を撤廃
pd.set_option('display.max_rows', None)

#列の表示数の上限を撤廃
pd.set_option('display.max_columns', None)


In [56]:
submit_df = pd.read_csv('./data/sample_submit.csv', header=None)

In [4]:
test_df = pd.read_csv('./data/test.csv')

In [5]:
train_df = pd.read_csv('./data/train.csv')

In [6]:
all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

In [7]:
# id:削除
all_df = all_df.drop("id", axis=1)

In [8]:
# Age:age_numericという数値型の特徴量に変換
# 年齢を表す特徴量に「代」「歳」「才」「漢数字＋歳」などの異なる形式が混在している場合、これらを全て数値型に変換する

# 漢数字を数値に変換する関数
def kanji2number(kanji_str):
    kanji_dict = {'〇': 0, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '十': 10}
    num = 0
    ten_multiplier = 1
    
    for char in reversed(kanji_str):
        if char == '十':
            ten_multiplier = 10
        else:
            num += kanji_dict.get(char, 0) * ten_multiplier
            ten_multiplier = 1
            
    return num

def convert_age_to_number(age_str):
    # 入力が文字列かどうかを確認
    if isinstance(age_str, str):
        # 1. 漢数字を数値に変換
        try:
            age_str = re.sub(r'[一二三四五六七八九十〇]+', lambda x: str(kanji2number(x.group())), age_str)
        except Exception:
            pass

        # 2. 数字と単位を取り出す
        match = re.search(r'(\d+)(代|歳|才)?', age_str)
        if match:
            number = int(match.group(1))
            return number  # 数字と単位があればそのまま返す
    elif isinstance(age_str, (int, float)):
        if math.isnan(age_str):  # NaNを確認
            return age_str
        return int(age_str)

    return None

In [9]:
def split_features(text):
    # 区切り文字の正規表現パターンを定義（全角スペースや半角スペースを考慮）
    delimiters = r'[ 　、/,\t／\n\\]+'
    
    # 正規表現を使用してテキストを分割
    features = re.split(delimiters, text, 2)
    
    # 空文字を取り除く
    features = [feature for feature in features if feature.strip()]
    
    return features

# 特徴量を分割して新しい列を作成
# 特徴量の数に応じて動的に列を作成する
def add_features_columns(df, feature_column):
    # 特徴量の分割
    split_data = df[feature_column].apply(split_features)
    
    # 最大の特徴量の数を取得
    max_features = split_data.apply(len).max()
    
    # 新しい列名を作成
    new_columns = [f'特徴{i+1}' for i in range(max_features)]
    
    # 各行の特徴量を新しい列に分割
    split_df = pd.DataFrame(split_data.tolist(), columns=new_columns)
    
    # 元のデータフレームに新しい列を追加
    df = df.join(split_df)
    
    return df

In [10]:
# 特徴２を自動車所持している時を１、していない時を０に分類
def categorize_car(status):
    # 車を所持していない場合のキーワード
    no_car_keywords = [
        "車未所持", "自動車未所有", "自家用車なし", 
        "乗用車なし", "車なし", "車保有なし"
    ]
    
    # 車を所持している場合のキーワード
    has_car_keywords = [
        "車所持", "自動車所有", "自家用車あり", 
        "乗用車所持", "車保有", "車あり"
    ]
    
    if status in no_car_keywords:
        return 0
    elif status in has_car_keywords:
        return 1
    else:
        return None  # 不明な場合はNoneを返す

In [11]:
# 特徴３の整理

# 分類用の関数
def classify_children(text):
    if "なし" in text or "無し" in text or "ゼロ" in text or "無子" in text or "非育児家庭" in text:
        return 0
    elif "1人" in text or "こども1人" in text or "1児" in text or "子供有り(1人)" in text or "子供有り 1人" in text:
        return 1
    elif "2人" in text or "こども2人" in text or "2児" in text or "子供有り 2人" in text or "子供有り(2人)" in text:
        return 2
    elif "3人" in text or "こども3人" in text or "3児" in text or "子供有り 3人" in text or "子供有り(3人)" in text:
        return 3
    elif "不明" in text or "わからない" in text or "不詳" in text:
        return -1
    else:
        return -1  # 該当しない場合は不明として扱う

In [12]:
# DurationOfPitch：数値型に変換

# 分と秒の混合形式の文字列を秒単位に変換
def convert_to_seconds(time_str):
    if pd.isna(time_str):
        return time_str  # 欠損値はそのまま返す

    if isinstance(time_str, str):
        minutes = 0
        seconds = 0

        if '分' in time_str:
            # 分を抽出
            parts = time_str.split('分')
            minutes = int(parts[0].strip())
            time_str = parts[1] if len(parts) > 1 else '0秒'

        if '秒' in time_str:
            # 秒を抽出
            seconds = int(time_str.split('秒')[0].strip())

        # 総秒数を計算
        total_seconds = minutes * 60 + seconds
        return total_seconds
    else:
        return time_str  # 文字列でない場合はそのまま返す

In [13]:
# Gender分類用の関数
def categorize_gender(status):
    # 車を所持していない場合のキーワード
    male_keywords = [
        "Male", "male", "MALE", 
        "Ｍａｌｅ", "ｍａｌｅ", "ＭＡＬＥ"
    ]
    
    # 車を所持している場合のキーワード
    female_keywords = [
        "Female", "female", "Fe Male", "FEMALE", "Ｆｅｍａｌｅ", "ｆｅｍａｌｅ",
        "fe male", "FE MALE", "Ｆｅ　Ｍａｌｅ", "ＦＥＭＡＬＥ", "ｆｅ　ｍａｌｅ", "ＦＥ　ＭＡＬＥ"
    ]
    
    if status in male_keywords:
        return 1
    elif status in female_keywords:
        return 0
    else:
        return None  # 不明な場合はNoneを返す

In [14]:
# ProductPitched分類用の関数
def categorize_Product(status):
    basic_keywords = [
        "Basic", "basic", "BASIC", "Basıc", "Βasic", "Basi𝘤",
        "Вasic", "𐊡asic", "Basiс", "B𝖺sic", "B𝖺sic", "𐊡asi𝘤",
        "Basıϲ", "Βasıc", "BASIС", "B𝖺si𝘤", "ΒASIС", "basiϲ",
        "В𝖺sic", "BAꓢIC", "BAՏIC", "BΑSIC", "Βası𝘤", "BASΙC",
        "Baｓic", "basıc", "Basiϲ", "Basiϲ", "Basiϲ"
    ]
    
    standard_keywords = [
        "Standard", "standard", "STANDARD", "Stand𝖺rd", "Տtandard", "Staոdard",
        "Standa𝘳d", "S𝘵andard", "Standar𝔡", "St𝖺ndard", "ꓢtandard", "staոdard",
        "Ѕtandard", "Տtanda𝘳d", "STANᗞARD", "STANDARᎠ", "ЅTANDARD"
    ]
    
    deluxe_keywords = [
        "Deluxe", "DELUXE", "deluxe", "De|uxe", "Delu×e", "ᗞeluxe", "𝙳eluxe",
        "DELUXΕ", "Ꭰeluxe", "de|u×e", "ᎠELUXE", "de|uxe", "DΕLUXΕ"
    ]
    
    super_deluxe_keywords = [
    "Super Deluxe", "super deluxe", "SUPER DELUXE", "Super De|uxe", "Super ᗞeluxe",
    "Super Ꭰeluxe", "ꓢuper Deluxe", "Ѕuper Deluxe", "Super 𝙳eluxe", "SUPER DΕLUXE",
    "ꓢuper De|uxe", "Տuper Deluxe ", "super de|uxe", "ｓuper deluxe", "Տuper Deluxe"
    ]
    
    king_keywords = [
    "King", "KING", "king", "Kıոg", "Kiոg", "Kıng"
    ]
    
    if status in basic_keywords:
        return 0
    elif status in standard_keywords:
        return 1
    elif status in deluxe_keywords:
        return 2
    elif status in super_deluxe_keywords:
        return 3
    elif status in king_keywords:
        return 4
    else:
        return None  # 不明な場合はNoneを返す

In [15]:
# Designation分類用の関数
def categorize_Product(status):
    executive_keywords = [
        "Executive", "Exеcutive", "Exеcutivе", "Executivе", "Executiѵe", "Execuｔive", "Exеcutiѵе",
        "Еxecutive", "Exеcｕtive", "Е×еcutive", "E×ecutive", "Executiѵе", "Еxecutivе", "E×ecｕtive",
        "Exеcｕtivе", "Еxеcutivе", "Execｕtive", "E×еcutiѵe", "Еxecuｔive", "Е×ecutive"
    ]
    
    manager_keywords = [
        "Manager", "Μanager", "Manαger", "Managеr", "Mαnager", "Manage𝙧", "Mαnage𝙧", "Mαnαger",
        "Mαnagеr", "Μanage𝙧", "Manαgеr", "Μanagеr"
    ]
    
    senior_manager_keywords = [
        "Senior Manager", "Senior Managеr", "Senior Manαger", "Sеnior Manager", "Senior Mαnαger",
        "Senior Manage𝙧", "Տenior Manager", "Ѕenior Manager", "Sеnior Managеr", "Senior Mαnager",
        "Senior Mαnager", "Ѕenior Manαger", "Տenior Μanager", "Senio𝙧 Manager", "Senior Managе𝙧",
        "Senior Μanαger"
    ]
    
    avp_keywords = [
    "AVP", "АVP", "ΑVP", "AVＰ"
    ]
    
    vp_keywords = [
    "VP", "VＰ"
    ]
    
    if status in executive_keywords:
        return 0
    elif status in manager_keywords:
        return 1
    elif status in senior_manager_keywords:
        return 2
    elif status in avp_keywords:
        return 3
    elif status in vp_keywords:
        return 4
    else:
        return None  # 不明な場合はNoneを返す

In [16]:
# NumberOfTrips分類用の関数
def categorize_NumberOfTrips(status):

    one = ["1", "年に1回"]
    
    two= ["2", "年に2回", "半年に1回"]
    
    three = ["3", "年に3回"]
    
    four = ["4", "年に4回", "四半期に1回"]
    
    five = ["5", "年に5回"]
    
    six = ["6", "年に6回"]
    
    seven = ["7", "年に7回"]

    eight = ["8", "年に8回"]
    
    if status in one:
        return 1
    elif status in two:
        return 2
    elif status in three:
        return 3
    elif status in four:
        return 4
    elif status in five:
        return 5
    elif status in six:
        return 6
    elif status in seven:
        return 7
    elif status in eight:
        return 8
    else:
        return None

In [17]:
# 関数を適用して新しい列を作成
all_df['Age'] = all_df['Age'].apply(convert_age_to_number)

# 特徴量列を分割して新しい列を作成
all_df = add_features_columns(all_df, 'customer_info')
all_df = all_df.drop('customer_info', axis=1)
all_df = all_df.rename(columns={'特徴1': '結婚歴', '特徴2': '車の有無', '特徴3': '子供の有無'}, copy=False)

all_df['車の有無'] = all_df['車の有無'].apply(categorize_car)

# データを分類
all_df['子供の有無'] = all_df['子供の有無'].apply(classify_children)

all_df['DurationOfPitch'] = all_df['DurationOfPitch'].apply(convert_to_seconds)

all_df['Gender'] = all_df['Gender'].apply(categorize_gender)

all_df['ProductPitched'] = all_df['ProductPitched'].apply(categorize_Product)

all_df['Designation'] = all_df['Designation'].apply(categorize_Product)

all_df['NumberOfTrips'] = all_df['NumberOfTrips'].apply(categorize_NumberOfTrips)

# MonthlyIncome分類用の関数
# 数値以外の文字を削除し、float型に変換
all_df['MonthlyIncome'] = all_df['MonthlyIncome'].str.replace('月収', '').str.replace('万円', '').astype(float) * 10000

In [18]:
all_df.head()

,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,結婚歴,車の有無,子供の有無
0,50.0,Self Enquiry,2,900.0,Large Business,1,1.0,4.0,None,3.0,5.0,1,4,0.0,2.539050e+09,1.0,未婚,0,0
1,56.0,Company Invited,1,840.0,Salaried,1,1.0,4.0,None,3.0,2.0,1,4,2.0,4.044750e+09,0.0,離婚済み,1,0
2,NaN,Self Enquiry,1,600.0,Large Business,0,1.0,3.0,None,3.0,4.0,0,4,0.0,2.781450e+09,1.0,結婚済み,0,0
3,37.0,Self Enquiry,2,1080.0,Small Business,0,1.0,3.0,None,4.0,1.0,0,5,2.0,3.268050e+09,0.0,離婚済み,1,0
4,48.0,Company Invited,3,1020.0,Small Business,0,1.0,3.0,None,4.0,4.0,0,4,0.0,2.584350e+09,1.0,独身,1,0


In [19]:
# One-Hot Encoding
all_df = pd.get_dummies(all_df, drop_first=True)

In [45]:
new_train_df = all_df[~all_df["ProdTaken"].isnull()]
new_test_df = all_df[all_df["ProdTaken"].isnull()].drop("ProdTaken",axis=1).reset_index(drop=True)


In [46]:
new_test_df.head()

,Age,CityTier,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,車の有無,子供の有無,TypeofContact_Self Enquiry,Occupation_Salaried,Occupation_Small Business,結婚歴_独身,結婚歴_結婚済み,結婚歴_離婚済み
0,48.0,2,780.0,1,1.0,4.0,3.0,7.0,0,3,3.0,4.969500e+09,1,0,True,False,True,False,True,False
1,30.0,2,720.0,0,1.0,4.0,3.0,4.0,1,3,2.0,3.000000e+05,0,0,True,False,True,False,True,False
2,25.0,1,540.0,0,1.0,4.0,3.0,1.0,0,3,0.0,2.600000e+05,0,0,True,True,False,False,False,True
3,21.0,2,420.0,1,1.0,4.0,4.0,1.0,0,3,2.0,2.598750e+09,1,0,False,True,False,False,False,True
4,41.0,1,420.0,1,1.0,4.0,3.0,1.0,0,4,0.0,2.688300e+09,1,0,False,True,False,True,False,False


In [22]:
train_X = new_train_df.drop(["ProdTaken"], axis=1)
train_Y = new_train_df["ProdTaken"]

In [23]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_Y, test_size=0.25, random_state=42)

In [106]:
"""
Training API：Scikit-learn APIより細かな設定が可能なため
Stratified K-Fold: クラス不均衡がある分類問題であるため
パラメータ調整：optuna
今回の評価指標であるroc_auc_scoreは確率からスコアを計算するため、model.predict_proba()を使用
（分類における予測方法には、予測を0と1の二値で判定するmodel.predict()と、確率を計算するmodel.predict_proba()の二つ）

"""
# パラメータ探索（Optuna）
def objective(trial):

    params = {
        'random_seed':42,
        'objective': 'binary',  # 目的に応じて変更 (binary, multiclass, etc.)
        'metric': 'auc',  # メトリックも目的に応じて変更
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_bin': trial.suggest_int("max_bin",50,200),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 256),
        'min_sum_hessian_in_leaf': trial.suggest_int('min_sum_hessian_in_leaf', 1, 10),        
#        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
#        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
#        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
#        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
#        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
#        'max_depth': trial.suggest_int('max_depth', 3, 8),
#        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    }

    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_eval = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

    model = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                      num_boost_round=100, early_stopping_rounds=100, verbose_eval=False)
    y_prob = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = np.round(y_prob)  # 0.5を閾値として0/1に分類
    score = roc_auc_score(np.round(y_val), np.round(y_pred))

    return score

In [107]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=42))
study.optimize(objective, n_trials=50)
study.best_params

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-09-01 17:26:59,057] A new study created in memory with name: no-name-481bab69-b6d3-4381-8edf-d1b689992207
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Fou

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002017 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 289
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000597 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 243
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

[I 2024-09-01 17:26:59,654] Trial 2 finished with value: 0.5899832775919732 and parameters: {'learning_rate': 0.02, 'num_leaves': 125, 'max_bin': 95, 'bagging_fraction': 0.45860326840383037, 'bagging_freq': 5, 'feature_fraction': 0.6640914962437607, 'min_data_in_leaf': 33, 'min_sum_hessian_in_leaf': 5}. Best is trial 0 with value: 0.5.
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('fe

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-09-01 17:27:07,332] Trial 48 finished with value: 0.5799832775919732 and parameters: {'learning_rate': 0.014, 'num_leaves': 133, 'max_bin': 173, 'bagging_fraction': 0.5547416962269639, 'bagging_freq': 2, 'feature_fraction': 0.8011859319546586, 'min_data_in_leaf': 238, 'min_sum_hessian_in_leaf': 6}. Best is trial 0 with value: 0.5.
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_80855/2822382652.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform(

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-09-01 17:27:07,574] Trial 49 finished with value: 0.5566555183946489 and parameters: {'learning_rate': 0.012, 'num_leaves': 51, 'max_bin': 67, 'bagging_fraction': 0.7663720254649795, 'bagging_freq': 3, 'feature_fraction': 0.7487429328535673, 'min_data_in_leaf': 41, 'min_sum_hessian_in_leaf': 5}. Best is trial 0 with value: 0.5.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [102]:
"""
# 得られたパラメータを設定(１回目)
lgbm_params = {
    'learning_rate': 0.018,
     'num_leaves': 125,
     'max_bin': 92,
     'bagging_fraction': 0.5064637262678338,
     'bagging_freq': 6,
     'feature_fraction': 0.8841008435603583,
     'min_data_in_leaf': 254,
     'min_sum_hessian_in_leaf': 5
}
"""

"# 得られたパラメータを設定(１回目)\nlgbm_params = {\n    'learning_rate': 0.018,\n     'num_leaves': 125,\n     'max_bin': 92,\n     'bagging_fraction': 0.5064637262678338,\n     'bagging_freq': 6,\n     'feature_fraction': 0.8841008435603583,\n     'min_data_in_leaf': 254,\n     'min_sum_hessian_in_leaf': 5\n}\n"

In [108]:
# optuna修正後
lgbm_params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    'metric': 'auc',
    'learning_rate': 0.01, 'num_leaves': 133, 'max_bin': 140, 'bagging_fraction': 0.8248435466776274, 'bagging_freq': 1, 'feature_fraction': 0.9819459112971965, 'min_data_in_leaf': 214, 'min_sum_hessian_in_leaf': 3
}

In [111]:
models = []
aucs = []
oof = np.zeros(len(train_X))

# クロスバリデーションの設定
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for train_index, val_index in skf.split(train_X, train_Y):
    X_train = train_X.iloc[train_index]
    X_val = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_val = train_Y.iloc[val_index]
    
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_eval = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

    model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                      num_boost_round=100, early_stopping_rounds=20, verbose_eval=10)
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred_label = np.rint(y_pred)  # 0.5を閾値として0/1に分類
    fpr, tpr, thresholds = roc_curve(y_val, y_pred)
    tmp_auc = auc(fpr, tpr)
    print(tmp_auc)
    
    models.append(model)
    aucs.append(tmp_auc)
    oof[val_index] = y_pred

/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 289
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/ishidzukamasato/.pyenv/versions/3.11.3/lib/python3.11/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_

In [112]:
# ROC曲線をプロット
plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

TypeError: must be real number, not function

In [113]:
sum(aucs)/len(aucs)

0.8006513592381344

In [114]:
# クロスバリデーション毎の各モデルで予測値を算出
preds = []

for model in models:
    pred = model.predict(new_test_df)
    preds.append(pred)
    
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)

In [115]:
# 予測値を元のスケールに戻す
# preds_exp = np.exp(preds_mean)

In [116]:
preds_mean

array([0.13333572, 0.13997995, 0.17507951, ..., 0.24897999, 0.17856732,
       0.10823259])

In [117]:
submit_df.head()

,0,1
0,3489,0.088587
1,3490,0.104305
2,3491,0.264710
3,3492,0.160500
4,3493,0.308525


In [118]:
submit_df[1] = preds_mean

In [119]:
submit_df.to_csv("./submit/submission_2.csv", index=False, header=None)

### 

In [43]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# パラメータの探索空間の定義
param_distributions = {
    'n_estimators': optuna.distributions.IntDistribution(2, 150),
    'max_depth': optuna.distributions.IntDistribution(1, 32, log=True)
}
# モデルの定義
clf = RandomForestClassifier()
# OptunaSearchCVの設定
optuna_search = OptunaSearchCV(
    clf,
    param_distributions,
    n_trials=100, # トライアル数（この例では100回）だけハイパーパラメータの組み合わせを試す
    scoring="accuracy",
    cv=cv,
    random_state=42,
    error_score='raise'
)
# モデルの最適化実行
optuna_search.fit(X_train, y_train)
# 最適化後のベストなパラメータとスコアの表示
print(f"Best parameters: {optuna_search.best_params_}")
print(f"Best cross-validation score: {optuna_search.best_score_:.3f}")
# テストデータに対する評価
test_score = optuna_search.score(X_val, y_val)
print(f"Test accuracy: {test_score:.3f}")
# 最適化されたモデルの特徴を取得して表示
best_estimator = optuna_search.best_estimator_
print(f"Best estimator: {best_estimator}")
# さらに、Optunaのスタディオブジェクトを取得して、詳細な最適化の結果を分析することも可能です。
study = optuna_search.study_
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: {trial.params}")
# Optunaのビジュアライゼーション機能を使って、探索の過程を視覚的に確認することもできます。
# 以下は、すべてのトライアルの結果をプロットする例です。
optuna.visualization.plot_optimization_history(study)

/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_89188/3044013357.py:11: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(
[I 2024-08-30 02:16:19,850] A new study created in memory with name: no-name-cc0586e9-4e25-4698-8045-32a556ae9987
[W 2024-08-30 02:16:19,859] Trial 0 failed with parameters: {'n_estimators': 69, 'max_depth': 16} because of the following error: ValueError('Input X contains NaN.\nRandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estima

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
"""
Training API：Scikit-learn APIより細かな設定が可能なため
Stratified K-Fold: クラス不均衡がある分類問題であるため
パラメータ調整：optuna
"""
# クロスバリデーションの設定
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# パラメータ探索（Optuna）
def objective(trial, data=X, target=y):

    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25, random_state=42)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        'objective': 'binary',  # 目的に応じて変更 (binary, multiclass, etc.)
        'metric': 'binary_logloss',  # メトリックも目的に応じて変更
        'boosting_type': 'gbdt',
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    }

    # クロスバリデーションによる評価
    cv_scores = []
    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

        model = lgb.train(param, train_data, valid_sets=[valid_data], early_stopping_rounds=100, verbose_eval=False)
        preds = model.predict(X_valid, num_iteration=model.best_iteration)
        pred_labels = np.rint(preds)  # 0.5を閾値として0/1に分類
        cv_scores.append(accuracy_score(y_valid, pred_labels))

    return np.mean(cv_scores)


In [ ]:
# Optunaによる最適化の実行
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 最適なハイパーパラメータ
best_params = study.best_params
print(f"Best Parameters: {best_params}")

# ベストパラメータでの最終モデルの学習
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X, y)

# モデルによる予測
predictions = final_model.predict(X)

In [ ]:
# kfoldの分割数
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

#目的変数
col = "ProdTaken"
#学習データの目的変数を設定
y = df[col]
#学習データの説明変数を設定（目的変数を除く）
X = df.drop(col, axis=1)
feature_names = df.columns.to_list()

auc_list = []
precision_list = []
recall_list = []

#lgbに渡す用のパラメータを設定
params = {
    # task: デフォルトは’train’。’train’, ‘prediction’, ‘refit’などから選択
    'task': 'train',
    # boosting_type: ‘gbdt’がデフォルトでGradient Boosting Decision Tree。他には’rf’とすることでランダムフォレストを選択する事も可能
    'boosting_type': 'gbdt',
    # objective: 何を目的としたモデルを訓練するのかを指定。’regression’がデフォルトで回帰。二値分類を目的とする’binary’や多クラス分類を行うための’multiclass’もある
    'objective': 'binary',
    # 訓練の指標とする損失関数を指定。二値分類は’binary_logloss’を選択。回帰のためのモデルだと’rmse’。多クラス分類だと’multi_logloss’などを指定
    'metric': 'binary_logloss',
    'num_leaves': 64,
    'min_data_in_leaf': 20,
    'max_depth': 7,
    # verbose: どれくらい詳しく訓練の途中経過を表示するかを指定
    'verbose': 0,
}

for train_index, test_index in tqdm(skf.split(X, y), total=k, desc="K-Fold Cross-Validation"):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: {}'.format(len(study.trials)))

    print('Best trial:')
    trial = study.best_trial

    print('  Value: {}'.format(trial.value))

    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))  
        
    # optunaでサーチしたパラメータ
    trial.params['objective'] = 'binary'
    lgbm_params = trial.params

    # ここではvalidをモデル評価、evalをフォールドアウト検証に使う・・・分割の大きさはデータセットと相談
    X_eval, X_valid, y_eval, y_valid = train_test_split(X_test, y_test, random_state=42, 
                                                        shuffle=True, stratify=y_test, test_size=0.2)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)

    # モデル評価用
    lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    #model = lgb.train(パラメータ, 学習データ, 検証データ, 学習木の数,学習が止まるまでの最大試行回数)
    model = lgb.train(params=lgbm_params, 
                      train_set=lgb_train,
                      valid_sets=lgb_valid,
                      num_boost_round=10000, #num_boost_round=1000:決定木を直列に結んで学習を行う回数
                      early_stopping_rounds=100)# early_stopping_rounds=100:過学習を防ぐため学習を打ち切るもの、100回行って推定精度があがらなかった場合とまる

    predict_proba = model.predict(X_test, num_iteration=model.best_iteration)
    predict = [0 if i < 0.5 else 1 for i in predict_proba]

    auc, precision, recall = get_evaluate(y_test, predict)

    print('AUC:{}, precision:{}, recall:{}'.format(auc, precision, recall))

    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)

# kfoldの平均値を取得
print('Kfold平均 AUC:{}, precision:{}, recall:{}'.format(np.mean(auc_list), 
                                                         np.mean(precision_list), 
                                                         np.mean(recall_list)))

[I 2024-08-14 19:46:03,233] A new study created in memory with name: no-name-73f7d64b-1fe6-46b0-9ed1-782a343db695
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_7949/2412603951.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0)
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_7949/2412603951.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  param['drop_rate'] = trial.suggest_loguniform('drop_rate', 1e-8, 1.0)
/var/folders/9w/zx7s1ndd7tx4ccd0jcmnz9wm0000gn/T/ipykernel_7949/2412603951.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be 

Number of finished trials: 100
Best trial:
  Value: 0.8825214899713467
  Params: 
    boosting: dart
    num_leaves: 368
    learning_rate: 0.05097448086895073
    drop_rate: 0.13412471855446875
    skip_drop: 0.0009530910150196744
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 409
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 21
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] No further splits with positive

In [ ]:
# 訓練したモデルをファイルとして保存
model.save_model('model.txt', num_iteration=bst.best_iteration)

In [ ]:
#検証データの予測値の算出
#予測値を返す
y_pred = model.predict(X_val, num_iteration=model.best_iteration)

roc = roc_curve(y_val, y_pred)

In [ ]:
#予測結果を出力
predict = model.predict(df_test)
df_test["取引価格(総額)_log"] = predict
df_test[["取引価格(総額)_log"]].to_csv("submit_test.csv")

In [ ]:
#特徴量重要度を示す
#feature_importance:それぞれの特徴量重要度が格納されている
#index=val_x.columns 検証データの説明変数が入っているval_xを持ってくる
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

#決定木の分岐に使われた回数を示すものであるため、数値間の単純比較はできない
#線形回帰モデルなら偏回帰係数で行える

In [ ]:
#別のpythonファイルで関数化したものを呼び出してデータ加工、モデル構築を行えるようにする
#テキストファイル(txt→py)に記述
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#新たな関数のインポート
import test_func as tf
%matplotlib inline

# 今後の方針
dropna
fillna
平均などで缺損値補完
標準化